In [ ]:
#import library

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
nltk.download('wordnet')
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import json


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#load data
with open('EnglishData.json') as file:
    data = json.load(file)

In [ ]:
# Extracting Data

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [ ]:
# Word Steamming
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

# Bag of Words
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

In [ ]:
# Split Data
train_data, val_data, train_output, val_output = train_test_split(training, output, test_size=0.25, random_state=42)

# Developing a Model
model = Sequential([
    Dense(32, input_shape=(len(training[0]),), activation='relu'),
    Dropout(0.4),
    Dense(16, activation='relu'),
    Dense(len(output[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history=model.fit(train_data, train_output, epochs=500, batch_size=32, validation_data=(val_data, val_output))

Epoch 1/500
5/5 [==============================] - 1s 73ms/step - loss: 3.8309 - accuracy: 0.0142 - val_loss: 3.8182 - val_accuracy: 0.0213
Epoch 2/500
5/5 [==============================] - 0s 15ms/step - loss: 3.8003 - accuracy: 0.0071 - val_loss: 3.8172 - val_accuracy: 0.0213
Epoch 3/500
5/5 [==============================] - 0s 15ms/step - loss: 3.8195 - accuracy: 0.0284 - val_loss: 3.8159 - val_accuracy: 0.0213
Epoch 4/500
5/5 [==============================] - 0s 11ms/step - loss: 3.7867 - accuracy: 0.0213 - val_loss: 3.8146 - val_accuracy: 0.0213
Epoch 5/500
5/5 [==============================] - 0s 12ms/step - loss: 3.7797 - accuracy: 0.0355 - val_loss: 3.8135 - val_accuracy: 0.0213
Epoch 6/500
5/5 [==============================] - 0s 15ms/step - loss: 3.7667 - accuracy: 0.0355 - val_loss: 3.8124 - val_accuracy: 0.0638
Epoch 7/500
5/5 [==============================] - 0s 11ms/step - loss: 3.7623 - accuracy: 0.0213 - val_loss: 3.8115 - val_accuracy: 0.0638
Epoch 8/500
5/5 [===

In [ ]:
import random
# Making Predictions
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return [bag]


def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict(bag_of_words(inp, words))
        results_index = np.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: how to cure vertigo
1/1 [==============================] - 0s 109ms/step
During a vertigo attack, lying still in a quiet, darkened room may help to ease any symptoms of nausea and reduce the sensation of spinning. You may be advised to take medication. You should also try to avoid stressful situations, as anxiety can make the symptoms of vertigo worse.
You: what to cure fever
1/1 [==============================] - 0s 40ms/step
To treat a fever at home: 1)Drink plenty of fluids to stay hydrated. 2)Dress in lightweight clothing. 3)Use a light blanket if you feel chilled, until the chills end. 4)Take acetaminophen (Tylenol, others) or ibuprofen (Advil, Motrin IB, others). 5) Get medical help if the fever lasts more than five days in a row.
You: quit


In [ ]:
model.save("EngChatbotModel.h5",history)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
